### 데이터 설명 입력 파트

In [ ]:
from typing import Annotated, TypedDict
from langgraph.graph import StateGraph, add_messages, END
from langgraph.checkpoint.memory import MemorySaver
from langchain_core.runnables import RunnableLambda
from langchain_core.messages import HumanMessage
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
import os

# .env에서 API 키 로드
load_dotenv()

# 상태 정의
class GraphState(TypedDict):
    question : Annotated[str, "question"]
    context : Annotated[str, "Context"]
    answer : Annotated[str, "Answer"]

# GPT 초기화
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 노드 1: 질문 처리
def handle_question(state: GraphState) -> GraphState:
    print("🔹 질문 처리 중:", state["question"])
    state["context"] = (
        "테이블명 : cust_info, 컬럼명 : [고객아이디,이름,나이,성별,주소,취미], "
        "컬럼 설명 : [고객아이디 : 고객의 아이디 정보, 이름 : 고객의 이름정보, "
        "나이 : 고객의 나이정보, 성별 : 고객의 성별 정보, 주소 : 고객의 집주소정보, 취미 : 고객의 취미정보]"
    )
    return state

# 노드 2: GPT로 답변 생성
def generate_answer(state: GraphState) -> GraphState:
    print("🔹 GPT API를 통해 답변 생성 중...")
    messages = [
        HumanMessage(content=f"질문: {state['question']}\n문맥: {state['context']}")
    ]
    response = llm.invoke(messages)
    state["answer"] = response.content
    return state

# 그래프 구성
workflow = StateGraph(GraphState)
workflow.add_node("handle_question", RunnableLambda(handle_question))
workflow.add_node("generate_answer", RunnableLambda(generate_answer))
workflow.set_entry_point("handle_question")
workflow.add_edge("handle_question", "generate_answer")
workflow.add_edge("generate_answer", END)
app = workflow.compile()


# 실행
input_state = {
    "question": "테이블 컬럼의 정보를 기반으로 카테고리형 컬럼이 무엇인지 알려줘. 답변은 컬럼명만 나열해줘.",
    "context": "",
    "answer": ""
}

result = app.invoke(input_state)
print("✅ 최종 결과:", result["answer"])

🔹 질문 처리 중: 테이블 컬럼의 정보를 기반으로 카테고리형 컬럼이 무엇인지 알려줘. 답변은 컬럼명만 나열해줘.
🔹 GPT API를 통해 답변 생성 중...
✅ 최종 결과: 성별, 취미


In [1]:
import pandas as pd
import numpy as np

In [ ]:
# 다음과 같은 형태로 동작하는 함수 만들어야함 - 테이블이 들어 오면 정규화해주는 함수
"""

def standardize_code(df: pd.DataFrame) -> pd.DataFrame:
    df_copy = df.copy()

    return df_copy


# 테스트 코드
if __name__ == "__main__":
    print("✅ 성별 정규화 테스트")
    df_customer = pd.read_csv("data/customer_info.csv")
    print(df_customer[["customer_id", "gender"]])
    df_std = standardize_gender(df_customer)
    print(df_std[["customer_id", "gender"]])

"""

In [34]:
# 도메인/코드 이상데이터 생성

import random
from faker import Faker

fake = Faker('ko_KR')  # 한국어 데이터 생성

def generate_gender():
    return random.choice([
        "남", "여", "남성", "여성", "M", "F", "Male", "Female", "남자", "여자", "man", "woman"
    ])

def generate_address():
    city = fake.city()
    district = fake.city_suffix()  # ~구 형태
    variations = [
        f"{city}시 {district}",
        f"{city}-{district}",
        f"{city} {district}",
        f"{district}, {city}",
        f"{city} {district}구",
        f"{district}({city})",
    ]
    return random.choice(variations)

def generate_hobby():
    hobby_variations = [
        "독서", "독서하기", "책 읽기", "독후감 작성",
        "운동", "헬스", "체력 단련", "피트니스",
        "영화 감상", "영화보기", "영화", "시네마",
        "등산", "산에 가기", "하이킹",
        "게임", "비디오게임", "온라인 게임",
        "요리", "쿠킹", "집밥 만들기"
    ]
    return random.choice(hobby_variations)

def generate_customer_data(num_rows=1000):
    data = []
    for _ in range(num_rows):
        cust_id = fake.unique.random_int(min=100000, max=999999)
        name = fake.name()
        age = random.randint(18, 80)
        gender = generate_gender()
        address = generate_address()
        hobby = generate_hobby()

        data.append([cust_id, name, age, gender, address, hobby])

    columns = ["고객아이디", "이름", "나이", "성별", "주소", "취미"]
    return pd.DataFrame(data, columns=columns)

# 데이터 생성
df = generate_customer_data()

df.to_csv("cust_info.csv", index=False)

# 결과 확인
print(df.head())

    고객아이디   이름  나이      성별      주소      취미
0  798313  김도현  21       여  동두천시 시      헬스
1  817100  최영진  25  Female  시, 수원시  독후감 작성
2  476331  김시우  59  Female   양양군 시  독후감 작성
3  487616  박성수  61    Male   포천시-시   산에 가기
4  650159  나지훈  44    Male  시(횡성군)    책 읽기


In [4]:
df = pd.read_csv('data/cust_info.csv')
df.head()

,고객아이디,이름,나이,성별,주소,취미
0,412772,신은영,69,여성,시(금산군),게임
1,538371,최현주,29,여성,천안시 서북구-시,온라인 게임
2,392401,김유진,23,woman,"시, 금산군",운동
3,758857,김광수,72,남,시흥시시 시,집밥 만들기
4,328613,강재현,43,man,남양주시 시구,등산


In [7]:
def col_uniq(colname):
    col_uniq_list = list(df[colname].unique())
    return col_uniq_list

col_uniq('성별')

['여성', 'woman', '남', 'man', '남성', 'F', '남자', '여', 'Female', 'Male', '여자', 'M']

In [5]:
df

,고객아이디,이름,나이,성별,주소,취미
0,412772,신은영,69,여성,시(금산군),게임
1,538371,최현주,29,여성,천안시 서북구-시,온라인 게임
2,392401,김유진,23,woman,"시, 금산군",운동
3,758857,김광수,72,남,시흥시시 시,집밥 만들기
4,328613,강재현,43,man,남양주시 시구,등산
...,...,...,...,...,...,...
995,537203,이준혁,18,F,강릉시 시,게임
996,601448,김은정,66,Female,부천시 원미구-시,독서
997,540915,김종수,28,man,"시, 금산군",온라인 게임
998,812926,강지혜,47,여성,"시, 서산시",쿠킹


In [8]:
import streamlit as st
import pandas as pd

st.title("컬럼 선택 후 분석 실행")

# 1. CSV 업로드
uploaded_file = st.file_uploader("CSV 파일 업로드", type=["csv"])
if uploaded_file is not None:
    df = pd.read_csv(uploaded_file)
    st.write("업로드된 데이터:")
    st.dataframe(df.head())

    # 2. 가정된 결과 리스트
    result = ['성별', '취미', '주소']  # 실제론 다른 코드에서 나온 값이라고 가정

    # 3. 체크박스 UI (멀티 선택)
    selected_columns = st.multiselect(
        "분석에 사용할 컬럼을 선택하세요",
        options=result,
        default=result  # 기본은 전부 선택된 상태
    )

    if selected_columns:
        # 4. 선택된 컬럼만 추출
        selected_df = df[selected_columns]

        st.subheader("선택된 컬럼의 데이터")
        st.dataframe(selected_df.head())

        # 5. 예시 분석 실행
        if st.button("분석 실행"):
            st.success(f"선택된 컬럼 {selected_columns}에 대해 분석을 시작합니다.")
            # 여기서 선택된 데이터로 실제 분석 함수 호출 가능
            # analyze(selected_df)
    else:
        st.warning("분석할 컬럼을 최소 1개 선택해주세요.")

2025-07-07 16:44:01.142 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.354 
  command:

    streamlit run c:\Users\user\anaconda3\Lib\site-packages\ipykernel_launcher.py [ARGUMENTS]
2025-07-07 16:44:01.355 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.356 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.357 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.358 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-07-07 16:44:01.358 Thread 'MainThread': mis